In [65]:
import numpy as np
import numpy.typing as npt
import matplotlib.pyplot as plt  # Biblioteca para gerar gráficos
import pandas as pd
from sklearn import metrics, model_selection
from scipy import stats
from scipy.spatial import distance
import math


In [66]:

class Set:
    def __init__(self, dataset, features, output):
        self.dataset = dataset
        self.features = features
        self.output = output

    def get_n(self):
        return self.dataset.shape[0]

    def get_x(self):
        return self.dataset[:, self.features]
    
    def get_x_apply(self, func):
        return func(self.dataset[:, self.features])

    def set_x(self, new_x):
        self.x = new_x

    def get_y(self):
        return self.dataset[:, self.output]

    def get_X(self, func=None):
        if (func):
            return np.c_[np.ones(self.get_n()), func(self.get_x())]
        else:
            return np.c_[np.ones(self.get_n()), self.get_x()]


In [67]:
def is_true_positive(y, y_pred):
    return y_pred >= 1 and y >= 1

def is_false_positive(y, y_pred):
    return y_pred >= 1 and y <= 0

def is_true_negative(y, y_pred):
    return y_pred <= 0 and y <= 0

def is_false_negative(y, y_pred):
    return y_pred <= 0 and y >= 1

def confusion_matrix(y, y_pred):
    """ returns (tp, fp, tn, fn) """

    tp, fp, tn, fn = 0, 0, 0, 0
    for i, pred in enumerate(y_pred):
        tp += 1 if is_true_positive(y[i], pred) else 0
        fp += 1 if is_false_positive(y[i], pred) else 0
        tn += 1 if is_true_negative(y[i], pred) else 0
        fn += 1 if is_false_negative(y[i], pred) else 0
    return (tp, fp, tn, fn)

def accuracy(y, y_pred):
    tp, fp, tn, fn = confusion_matrix(y, y_pred)
    return (tp + tn) / (tp + fp + tn + fn)

def precision(y, y_pred):
    tp, fp, tn, fn = confusion_matrix(y, y_pred)
    return tp / (tp + fp)

def recall(y, y_pred):
    tp, fp, tn, fn = confusion_matrix(y, y_pred)
    return tp / (tp + fn)

def f1_score(y, y_pred):
    precision_ = precision(y, y_pred)
    recall_ = recall(y, y_pred)
    return 2 * (precision_ * recall_) / (precision_ + recall_)

def get_metrics(n_folds):
    return {
        "accuracy": np.zeros(n_folds),
        "precision": np.zeros(n_folds),
        "recall": np.zeros(n_folds),
        "f1_score": np.zeros(n_folds)
    }

def calculate_metrics(metrics, y_test, y_pred):
    metrics["accuracy"][i] = (accuracy(y_test, y_pred))
    metrics["precision"][i] = (precision(y_test, y_pred))
    metrics["recall"][i] = (recall(y_test, y_pred))
    metrics["f1_score"][i] = (f1_score(y_test, y_pred))

def print_metrics(metrics, name, n_folds):
    print("%i-fold cross validation com %s" % (n_folds, name))
    print("acurácia: %.8f +/- %.8f" % (metrics["accuracy"].mean(), metrics["accuracy"].std()))
    print("revocação: %.8f +/- %.8f" % (metrics["precision"].mean(), metrics["precision"].std()))
    print("precisão: %.8f +/- %.8f" % (metrics["recall"].mean(), metrics["recall"].std()))
    print("f1-score: %.8f +/- %.8f" % (metrics["f1_score"].mean(), metrics["f1_score"].std()))
    print("")


In [68]:
def k_fold_split(array, k = int):
    """realiza o split dos dados em k-folds"""
    shuffled_data = np.random.permutation(array)
    folds = np.array_split(shuffled_data, k)
    return folds

def k_fold_train_test(folds):
    """retorna um vetor com as configurações de treino e teste definidas pelo k-fold split

    returns (i, train, test)
    """
    results = []
    for i, fold in enumerate(folds):
        train = np.vstack([x for j, x in enumerate(folds) if j != i])
        test = fold
        results.append((i, train, test))
    return results

# Questão 1

Considere o conjunto de dados disponível em **kc2.csv**, organizado em 22 colunas, sendo as 21 primeiras colunas os atributos e a última coluna a saída. Os 21 atributos são referentes à caracterização de códigos-fontes para processamento de dados na NASA. A saída é a indicação de ausência (0) ou existência (1) de defeitos. Maiores detalhes sobre os dados p o dem ser conferidos em https://www.openml.org/d/1063.

In [69]:
data = np.genfromtxt('kc2.csv', delimiter=',')
np.random.seed(666)
folds = k_fold_split(data, 10)
n_folds = len(folds)
features = np.arange(21)
output = 21

a) Considerando uma validação cruzada em 10 folds, avalie modelos de classificação binária nos dados em questão. Para tanto, use as abordagens abaixo:

- **KNN** (escolha k = 1 e k = 5, distância Euclidiana (e Mahalonobis, para a pós-graduação));
- **Árvore de decisão** (você pode usar uma implementação já existente com índices de impureza de gini e entropia)

In [70]:
def euclidean_distance(p, q):
    return np.abs(np.sqrt(np.sum((p - q)**2)))

def cov(X):
    mu = np.mean(X, axis=0)
    aux = X - mu
    return (aux).T.dot(aux) / (X.shape[0])

def inverse_cov(X):
    cov_ = cov(X)
    return np.linalg.pinv(cov_)

def mahalanobis_distance(x, u, vi):
    """vi: inversa da matriz de covariância dos dados de treino"""
    rs = (x - u).T
    rs = rs.dot(vi)
    rs = rs.dot((x - u))
    return np.sqrt(rs)

def kNN(X, y, x_test, y_test, k, metric = 'euclidean'):
    y_pred = []
    vi = inverse_cov(X)

    # encontrar k padrões mais próximos
    for xi in x_test:
        distances = []
        for j in range(X.shape[0]):
            if metric == 'euclidean':
                distances.append(euclidean_distance(xi, X[j, :]))
            elif metric == 'mahalanobis':
                distances.append(mahalanobis_distance(xi, X[j, :], vi))
            else:
                distances.append(euclidean_distance(xi, X[j, :]))
        distances = np.array(distances)

        dist = np.argsort(distances)[:k]
        labels = y[dist]

        lab = stats.mode(labels) 
        lab = lab.mode[0]
        y_pred.append(lab)

    # computar predição com base na ponderação das saidos dos padrões mais próximos
    return (y_test, np.array(y_pred))

b) Para cada modelo criado, reporte **valor médio** e **desvio padrão** das métricas de **acurácia**, **revocação**, **precisão** e **F1-score**.

In [71]:
models = [
    { 'name': 'euclidean', 'k': 1 },
    { 'name': 'euclidean', 'k': 5 },
    { 'name': 'mahalanobis', 'k': 1 },
    { 'name': 'mahalanobis', 'k': 5 },
]

for model in models:
    model_metrics = get_metrics(n_folds)
    for i, train, test in k_fold_train_test(folds):
        train = Set(train, features, output)
        test = Set(test, features, output)
        y_test, y_pred = kNN(train.get_x(), train.get_y(), test.get_x(), test.get_y(), model['k'], model['name'])
        calculate_metrics(model_metrics, y_test, y_pred)
    print_metrics(model_metrics, "kNN (k = %s) - %s" % (model['k'], model['name']), n_folds)

10-fold cross validation com kNN (k = 1) - euclidean
acurácia: 0.77579826 +/- 0.04814430
revocação: 0.45108947 +/- 0.15521950
precisão: 0.41541681 +/- 0.15497358
f1-score: 0.42191653 +/- 0.13682793

10-fold cross validation com kNN (k = 5) - euclidean
acurácia: 0.82155298 +/- 0.05290241
revocação: 0.58277778 +/- 0.18122823
precisão: 0.44840132 +/- 0.22814770
f1-score: 0.47842176 +/- 0.18875699

10-fold cross validation com kNN (k = 1) - mahalanobis
acurácia: 0.79869376 +/- 0.04780905
revocação: 0.51771645 +/- 0.14089937
precisão: 0.39297591 +/- 0.15838524
f1-score: 0.42838302 +/- 0.11797260

10-fold cross validation com kNN (k = 5) - mahalanobis
acurácia: 0.84092888 +/- 0.05658102
revocação: 0.70702381 +/- 0.17814190
precisão: 0.40205100 +/- 0.15967510
f1-score: 0.49589457 +/- 0.15256657



In [72]:
from sklearn.tree import DecisionTreeClassifier

criterions = ['gini', 'entropy']

for criterion in criterions:
    model_metrics = get_metrics(n_folds)
    for i, train, test in k_fold_train_test(folds):
        train = Set(train, features, output)
        test = Set(test, features, output)

        tree = DecisionTreeClassifier(criterion=criterion)
        tree.fit(train.get_x(), train.get_y())

        y_test, y_pred = test.get_y(), tree.predict(test.get_x())
        calculate_metrics(model_metrics, y_test, y_pred)
    print_metrics(model_metrics, "Decision Trees (%s)" % (criterion), n_folds)

10-fold cross validation com Decision Trees (gini)
acurácia: 0.80446299 +/- 0.04886414
revocação: 0.52933761 +/- 0.09637387
precisão: 0.50494172 +/- 0.12365909
f1-score: 0.50722794 +/- 0.08679636

10-fold cross validation com Decision Trees (entropy)
acurácia: 0.79303338 +/- 0.04136213
revocação: 0.47794594 +/- 0.16782376
precisão: 0.48169136 +/- 0.17583641
f1-score: 0.47438158 +/- 0.15938129

